In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\

In [2]:
# Import Dataset
file_path_21 = Path("Resources/batting_stats_2021.csv")
file_path_20 = Path("Resources/batting_stats_2020.csv")
file_path_19 = Path("Resources/batting_stats_2019.csv")
file_path_18 = Path("Resources/batting_stats_2018.csv")
file_path_17 = Path("Resources/batting_stats_2017.csv")
file_path_16 = Path("Resources/batting_stats_2016.csv")
file_path_15 = Path("Resources/batting_stats_2015.csv")
file_path_14 = Path("Resources/batting_stats_2014.csv")
file_path_13 = Path("Resources/batting_stats_2013.csv")
file_path_12 = Path("Resources/batting_stats_2012.csv")
file_path_11 = Path("Resources/batting_stats_2011.csv")
file_path_10 = Path("Resources/batting_stats_2010.csv")

# Read into Dataset
stats_21_df = pd.read_csv(file_path_21, index_col = 0)
stats_20_df = pd.read_csv(file_path_20, index_col = 0)
stats_19_df = pd.read_csv(file_path_19, index_col = 0)
stats_18_df = pd.read_csv(file_path_18, index_col = 0)
stats_17_df = pd.read_csv(file_path_17, index_col = 0)
stats_16_df = pd.read_csv(file_path_16, index_col = 0)
stats_15_df = pd.read_csv(file_path_15, index_col = 0)
stats_14_df = pd.read_csv(file_path_14, index_col = 0)
stats_13_df = pd.read_csv(file_path_13, index_col = 0)
stats_12_df = pd.read_csv(file_path_12, index_col = 0)
stats_11_df = pd.read_csv(file_path_11, index_col = 0)
stats_10_df = pd.read_csv(file_path_10, index_col = 0)

stats_21_df.head()

,Team,Pos,Age,G,AB,R,H,2B,3B,HR,...,CS,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS
Player,,,,,,,,,,,,,,,,,,,,,
Isiah Kiner-Falefa,TEX,SS,26,97,388,46,100,17,2,6,...,2,13,66,1,2,7,0.258,0.293,0.358,0.651
Tommy Edman,STL,2B,26,96,387,50,99,20,3,6,...,4,22,53,1,1,3,0.256,0.300,0.370,0.670
Bo Bichette,TOR,SS,23,93,380,73,110,18,1,16,...,0,25,86,0,1,4,0.289,0.339,0.468,0.807
Adam Frazier,PIT,2B,29,96,377,56,123,27,4,4,...,4,34,45,0,1,6,0.326,0.390,0.451,0.841
Whit Merrifield,KC,2B,32,95,377,54,103,21,2,8,...,1,28,56,0,9,3,0.273,0.321,0.403,0.724


In [3]:
# Creating X & y variables
X = stats_21_df.iloc[:, 4:22]
y = stats_21_df["OBP"].values

In [4]:
# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [5]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=78)

In [6]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [7]:
# Model set-up
number_input_features = 19
hidden_nodes_layer1 = 38
hidden_nodes_layer2 = 5

In [8]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="sigmoid"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
# Training the model
batch_size = 1000
epochs = 50
training_history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

ValueError: Error when checking input: expected dense_input to have 2 dimensions, but got array with shape (358, 18, 1)

In [ ]:
# Make predictions using the testing data X_test
predicted = model.predict(X_test)